<a href="https://colab.research.google.com/github/gibsonx/CE888/blob/master/Assignment/Assignment_2_xlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow tensorflow_datasets tensorflow_text zhon sentencepiece transformers focal-loss tfa-nightly
!git clone https://github.com/cardiffnlp/tweeteval.git

In [ ]:
import os,pathlib,re,nltk,pickle,string
nltk.download('punkt')

import tensorflow_addons as tfa
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text as text
from tensorflow.keras import Input, layers, losses, preprocessing, utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers.core import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from zhon import hanzi 
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import csv
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize
from focal_loss import BinaryFocalLoss
# import tokenization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from sklearn.feature_extraction.text import TfidfVectorizer
# import tensorflow_hub as hub
# import tensorflow_datasets as tfds
# from collections import Counter
# from nltk.stem import WordNetLemmatizer
# from google.colab import output
# from google.colab import drive
from sklearn.metrics import accuracy_score,confusion_matrix
from zhon import hanzi 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import csv
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.metrics import f1_score
try:
    %tensorflow_version 2.x
except Exception:
    pass
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# drive.mount('/content/drive/')
base_dir = '/content/tweeteval/datasets/'
hate_dir = base_dir + "hate"
irony_dir = base_dir + "irony"
offensive_dir = base_dir + "offensive"

In [ ]:
def imbalance_under_sampling(dfname):
  df_label_a = dfname[dfname['label'] == 1]
  df_label_b = dfname[dfname['label'] == 0]
  if df_label_a.shape[0] > df_label_b.shape[0]:
  # count_class_0, count_class_1 = dfname.label.value_counts()
    df_label_a = df_label_a.sample(df_label_b.shape[0],random_state=1)
    df = pd.concat([df_label_b, df_label_a], axis=0)
    print('label 0 is more',df.label.value_counts())  
    return df
  else: 
    df_label_b = df_label_b.sample(df_label_a.shape[0],random_state=1)
    df = pd.concat([df_label_b, df_label_a], axis=0)
    print('label 1 is more',df.label.value_counts())  
    return df

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)

def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    # Remove urls
    tweet = re.sub(r"^#\S+|\s#\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r"^@\S+|\s@\S+",'', tweet)
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = tweet.translate(str.maketrans('', '', hanzi.punctuation))
    tweet = remove_emoji(tweet)
    # tweet_tokens = word_tokenize(tweet)
    # # Remove stopwords
    # filtered_words = [w for w in tweet_tokens]
    # return " ".join(filtered_words)
    return tweet

def readfile(text):
  pd_list = []
  with open(text,'r') as f:
    for tweet in f.read().splitlines():
      tweet = preprocess_tweet_text(tweet)
      pd_list.append(tweet)   
  return pd_list
 
def readfile_label(text):
  pd_list = []
  with open(text,'r') as f:
    for tweet in f.read().splitlines():
      pd_list.append(int(tweet))  
  return pd_list

In [ ]:
#Pre-process csv and merge into a Dataframe
#Hate DataFrame Data 
hate_dict_train = {'text': readfile(os.path.join(hate_dir,"train_text.txt")),'label': readfile_label(os.path.join(hate_dir,"train_labels.txt"))}
hate_dict_val = {'text': readfile(os.path.join(hate_dir,"val_text.txt")),'label': readfile_label(os.path.join(hate_dir,"val_labels.txt"))}
hate_dict_test = {'text': readfile(os.path.join(hate_dir,"test_text.txt")),'label': readfile_label(os.path.join(hate_dir,"test_labels.txt"))}
hate_train_df = imbalance_under_sampling(pd.DataFrame(hate_dict_train))
hate_val_df = pd.DataFrame(hate_dict_val)
hate_test_df = pd.DataFrame(hate_dict_test)
hate_frames = [hate_train_df, hate_val_df, hate_test_df]
df_hate = pd.concat(hate_frames)
not_hate = df_hate[df_hate['label'] == 0]
hate = df_hate[df_hate['label'] == 1]

#Irony DataFrame
irony_dict_train = {'text': readfile(os.path.join(irony_dir,"train_text.txt")),'label': readfile_label(os.path.join(irony_dir,"train_labels.txt"))}
irony_dict_val = {'text': readfile(os.path.join(irony_dir,"val_text.txt")),'label': readfile_label(os.path.join(irony_dir,"val_labels.txt"))}
irony_dict_test = {'text': readfile(os.path.join(irony_dir,"test_text.txt")),'label': readfile_label(os.path.join(irony_dir,"test_labels.txt"))}
irony_train_df = imbalance_under_sampling(pd.DataFrame(irony_dict_train))
irony_val_df = pd.DataFrame(irony_dict_val)
irony_test_df = pd.DataFrame(irony_dict_test)
irony_frames = [irony_train_df,irony_val_df,irony_test_df]
df_irony = pd.concat(irony_frames)
not_irony  = df_irony[df_irony['label'] == 0]
irony  = df_irony[df_irony['label'] == 1]

#Offensive DataFrame
offensive_dict_train = {'text': readfile(os.path.join(offensive_dir,"train_text.txt")),'label': readfile_label(os.path.join(offensive_dir,"train_labels.txt"))}
offensive_dict_val = {'text': readfile(os.path.join(offensive_dir,"val_text.txt")),'label': readfile_label(os.path.join(offensive_dir,"val_labels.txt"))}
offensive_dict_test = {'text': readfile(os.path.join(offensive_dir,"test_text.txt")),'label': readfile_label(os.path.join(offensive_dir,"test_labels.txt"))}
offensive_train_df = imbalance_under_sampling(pd.DataFrame(offensive_dict_train))
offensive_val_df = pd.DataFrame(offensive_dict_val)
offensive_test_df = pd.DataFrame(offensive_dict_test)
offensive_frames = [offensive_train_df,offensive_val_df,offensive_test_df]
df_offensive = pd.concat(offensive_frames)
not_offensive  = df_offensive[df_offensive['label'] == 0]
offensive  = df_offensive[df_offensive['label'] == 1]

hate = df_hate[df_hate['label'] == 1]
hate

In [ ]:
from transformers import TFXLNetModel, XLNetTokenizer
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
xlnet_model = TFXLNetModel.from_pretrained('xlnet-large-cased')

In [ ]:
def create_xlnet(xl_model):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_ids = tf.keras.Input(shape=(120,), name='word_ids', dtype='int32')
    word_attention = tf.keras.Input(shape=(120,), name='word_attention', dtype='int32')
    # word_seq = tf.keras.Input(shape=(120,), name='word_seq', dtype='int32')

    # Call XLNet model
    xlnet_encodings = xl_model([word_ids,word_attention])[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    # Apply dropout for regularization
    dense = tf.keras.layers.Dense(32, activation='relu', name='encoding')(doc_encoding)
    drop = tf.keras.layers.Dropout(0.1)(dense)
    # Final output 
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(drop)

    # Compile model
    model = tf.keras.Model(inputs=[word_ids,word_attention], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss=BinaryFocalLoss(gamma=2), metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [ ]:
def get_inputs(tweets, tokenizer, max_len=120):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in tweets]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    return inp_tok, ids

def plot_metrics(pred, true_labels):
    """Plots a ROC curve with the accuracy and the AUC"""
    acc = accuracy_score(true_labels, np.array(pred.flatten() >= .5, dtype='int'))
    fpr, tpr, thresholds = roc_curve(true_labels, pred)
    auc = roc_auc_score(true_labels, pred)

    fig, ax = plt.subplots(1, figsize=(8,8))
    ax.plot(fpr, tpr, color='red')
    ax.plot([0,1], [0,1], color='black', linestyle='--')
    ax.set_title(f"AUC: {auc}\nACC: {acc}");
    return fig

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, min_delta=0.001, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=1e-6, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-6)
]    

In [ ]:
def plot_confusion_matrix(cm,target_names,title='Confusion matrix',cmap=None,normalize=True):
  accuracy = np.trace(cm) / float(np.sum(cm))
  misclass = 1 - accuracy

  if cmap is None:
      cmap = plt.get_cmap('Blues')

  plt.figure(figsize=(8, 6))
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()

  if target_names is not None:
      tick_marks = np.arange(len(target_names))
      plt.xticks(tick_marks, target_names, rotation=45)
      plt.yticks(tick_marks, target_names)

  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


  thresh = cm.max() / 1.5 if normalize else cm.max() / 2
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      if normalize:
          plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
      else:
          plt.text(j, i, "{:,}".format(cm[i, j]),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
  plt.show()

In [ ]:
hate_train_input = get_inputs(hate_train_df['text'], xlnet_tokenizer)
hate_val_input = get_inputs(hate_val_df['text'], xlnet_tokenizer)
hate_test_input = get_inputs(hate_test_df['text'], xlnet_tokenizer)

In [ ]:
xlnet_hate = create_xlnet(xlnet_model)
xlnet_hate.summary()
hist = xlnet_hate.fit(x=hate_train_input, y=hate_train_df['label'], epochs=20, batch_size=16, validation_data=(hate_val_input, hate_val_df['label']), callbacks=callbacks)

In [ ]:
preds = xlnet_hate.predict(hate_test_input, verbose=True)
y_pred = [i[0] for i in preds.round().astype(int)]
cm = confusion_matrix(hate_test_df['label'],y_pred)
plot_confusion_matrix(cm, normalize=False,target_names=['Not_Hate', 'hate'],title="Confusion Matrix for hate")

In [ ]:
f1_score(hate_test_df.label,y_pred,average='macro')

In [ ]:
irony_train_input = get_inputs(irony_train_df['text'], xlnet_tokenizer)
irony_val_input = get_inputs(irony_val_df['text'], xlnet_tokenizer)
irony_test_input = get_inputs(irony_test_df['text'], xlnet_tokenizer)

In [ ]:
xlnet_irony = create_xlnet(xlnet_model)
hist = xlnet_irony.fit(x=irony_train_input, y=irony_train_df['label'], epochs=2, batch_size=16, validation_data=(irony_val_input, irony_val_df['label']), callbacks=callbacks)

In [ ]:
preds = xlnet_irony.predict(irony_test_input, verbose=True)
y_pred = [i[0] for i in preds.round().astype(int)]
cm = confusion_matrix(irony_test_df['label'],y_pred)
plot_confusion_matrix(cm, normalize=False,target_names=['Not_irony ', 'irony'],title="Confusion Matrix for irony")

In [ ]:
f1_score(irony_test_df.label,y_pred,average='macro')

In [ ]:
offensive_train_input = get_inputs(offensive_train_df['text'], xlnet_tokenizer)
offensive_val_input = get_inputs(offensive_val_df['text'], xlnet_tokenizer)
offensive_test_input = get_inputs(offensive_test_df['text'], xlnet_tokenizer)

In [ ]:
xlnet = create_xlnet(xlnet_model)

In [ ]:
hist = xlnet.fit(x=offensive_train_input, y=offensive_train_df['label'], epochs=3, batch_size=16, validation_data=(offensive_val_input, offensive_val_df['label']), callbacks=callbacks)

In [ ]:
preds = xlnet.predict(offensive_test_input, verbose=True)

In [ ]:
# plot_metrics(preds, y_test)
y_pred = [i[0] for i in preds.round().astype(int)]
cm = confusion_matrix(offensive_test_df['label'],y_pred)
plot_confusion_matrix(cm, normalize=False,target_names=['Not_offensive', 'offensive'],title="Confusion Matrix for offensive")

In [ ]:
xlnet.evaluate(offensive_test_input,offensive_test_df['label'], verbose=True)

In [ ]:
f1_score(offensive_test_df.label,y_pred,average='macro')